# IBMi Jupyter Notebook Extension

In [ ]:
import pyodbc, getpass
import pandas as pd
from IPython.display import display
from IPython.core.magic import register_cell_magic

global _config
global _info

_config = {
    'host': '',
    'user': '',
    'pwd': ''
}

_info = {
    'sqlcode': 0,
    'sqlstate': '0',
    'sqlerror': ''
}

# disable pandas df display limit, trust user to limit result set in query
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

_config['host'] = input('Enter host: ').strip()
_config['user'] = input('Enter user: ').strip()
_config['pwd'] = getpass.getpass('Enter password: ')

In [ ]:
print("Connecting to {} as {} ...".format(_config['host'], _config['user']))
try:
    conn = pyodbc.connect(driver='{IBM i Access ODBC Driver}', 
      system=_config['host'], uid=_config['user'], pwd=_config['pwd'])
    print('Successfully connected!')
except pyodbc.InterfaceError as e:
    print('Could not connect :(\n{}'.format(e))

In [ ]:
# Add IBMi DB2 SQL magic: 
# invoke with %%ibmi
@register_cell_magic
def ibmi(line, cell):
    if conn is None:
        print('Not connected.')
        return None
    sql = ' '.join([x.replace('{','{{').replace('}','}}').replace('\n','').strip() for x in cell.split('\n')])
    if len(sql) > 0:
        print('Executing...')
        sql = sql.format(**globals())
        if line.strip() != '-':
            try:
                return pd.read_sql(sql, conn)
            except TypeError:
                print('Statement executed successfully')
        else:
            conn.execute(sql)
    else:
        print('Cannot execute blank query')
del ibmi